In [2]:
import torch
import model

In [3]:
model = model.ConvNet()
model.load_state_dict(torch.load("model.pkl", map_location='cpu'))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [4]:
for name,parameters in model.named_parameters():
    print(name,':',parameters.size())

features.0.weight : torch.Size([16, 1, 3, 3])
features.0.bias : torch.Size([16])
features.2.weight : torch.Size([32, 16, 3, 3])
features.2.bias : torch.Size([32])
features.5.weight : torch.Size([32, 32, 3, 3])
features.5.bias : torch.Size([32])
classifier.0.weight : torch.Size([128, 6272])
classifier.0.bias : torch.Size([128])
classifier.2.weight : torch.Size([10, 128])
classifier.2.bias : torch.Size([10])


In [29]:
for name, sub_module in model.named_modules():
#     print(name, ':', sub_module)
    if type(sub_module) is torch.nn.Conv2d:
        print(name)
#         print(sub_module.in_channels)
#         print(sub_module.padding)
#         print(sub_module.stride)
        print(sub_module.kernel_size[0])

features.0
3
features.2
3
features.5
3


In [48]:
def write_define(define_name, define_value, config_file):
    config_file.write("#define "+str(define_name)+" "+str(define_value)+"\n")
    
def generateConfig(model, in_row, in_col, path_to_config):
    config_file = open(path_to_config, "w")
    cov_cnt = 0
    pool_cnt = 0
    linear_cnt = 0
    in_ch = 0
    for name, sub_module in model.named_modules():
        if type(sub_module) is torch.nn.Conv2d:
            write_define("COV_" + str(cov_cnt) + "_IN_CH", sub_module.in_channels, config_file)
            write_define("COV_" + str(cov_cnt) + "_IN_ROW", in_row, config_file)
            write_define("COV_" + str(cov_cnt) + "_IN_COL", in_col, config_file)
            
            in_row = (in_row + 2 * sub_module.padding[0] - sub_module.kernel_size[0]) // sub_module.stride[0] + 1
            in_col = (in_col + 2 * sub_module.padding[0] - sub_module.kernel_size[0]) // sub_module.stride[0] + 1
            in_ch = sub_module.out_channels
            write_define("COV_" + str(cov_cnt) + "_OUT_CH", sub_module.out_channels, config_file)
            write_define("COV_" + str(cov_cnt) + "_OUT_ROW", in_row, config_file)
            write_define("COV_" + str(cov_cnt) + "_OUT_COL", in_col, config_file)
            
           
            write_define("COV_" + str(cov_cnt) + "_K", sub_module.kernel_size[0], config_file)
            write_define("COV_" + str(cov_cnt) + "_S", sub_module.stride[0], config_file)
            write_define("COV_" + str(cov_cnt) + "_P", sub_module.padding[0], config_file)
            if sub_module.bias[0] == True:
                write_define("COV_" + str(cov_cnt) + "_B", 1, config_file)
            else:
                write_define("COV_" + str(cov_cnt) + "_B", 0, config_file)
            cov_cnt = cov_cnt + 1
            config_file.write("\n")
        elif type(sub_module) is torch.nn.MaxPool2d:
            write_define("POOL_" + str(pool_cnt) + "_IN_CH", in_ch, config_file)
            write_define("POOL_" + str(pool_cnt) + "_IN_ROW", in_row, config_file)
            write_define("POOL_" + str(pool_cnt) + "_IN_COL", in_col, config_file)
            write_define("POOL_" + str(pool_cnt) + "_IN_PO", sub_module.kernel_size, config_file)
            in_row = in_row // sub_module.kernel_size
            in_col = in_col // sub_module.kernel_size
            pool_cnt = pool_cnt + 1
            config_file.write("\n")
        elif type(sub_module) is torch.nn.Linear:
            write_define("LINEAR_" + str(linear_cnt) + "_IN_N", sub_module.in_features, config_file)
            write_define("LINEAR_" + str(linear_cnt) + "_OUT_N", sub_module.out_features, config_file)
            linear_cnt = linear_cnt + 1
            config_file.write("\n")
#         elif type(sub_module) is torch.nn.ReLU:
#             write_define("LINEAR_" + str(linear_cnt) + "_IN_N", sub_module.in_features, config_file)
    config_file.close()

In [50]:
generateConfig(model, 28, 28, 'config.h')

In [57]:
for name, sub_module in model.named_modules():
    if type(sub_module) is torch.nn.Conv2d:
        print(sub_module.weight.size())
        print(sub_module.bias.size())

torch.Size([16, 1, 3, 3])
torch.Size([16])
torch.Size([32, 16, 3, 3])
torch.Size([32])
torch.Size([32, 32, 3, 3])
torch.Size([32])


In [91]:
# import numpy np

# import numpy as np  #借助numpy模块的set_printoptions()函数，将打印上限设置为无限即可

def generate_params(model, path_to_config):
#     np.set_printoptions(threshold=np.inf)
    torch.set_printoptions(threshold=np.inf)
    config_file = open(path_to_config, "w")
    cov_cnt = 0
    linear_cnt = 0

    for name, sub_module in model.named_modules():
        if type(sub_module) is torch.nn.Conv2d:
            w = sub_module.weight.detach()
#             print(sub_module.weight.detach().size())
            str_h = 'const float conv_' + str(cov_cnt) + '_w' + '[' + str(w.size()[0]) + ']' + '[' + str(w.size()[1]) + ']' \
            + '[' + str(w.size()[2]) + ']' + '[' + str(w.size()[3]) + '] = \n' 
            str_b = str(w)
#             print(str_b)
            str_b = str_b.replace('tensor(', '')
            str_b = str_b.replace(')', '')
            str_b = str_b.replace('[', '{')
            str_b = str_b.replace(']', '}')
            str1 = str_h + str_b + ';\n'
            
            config_file.write(str1)
            
            
            b = sub_module.bias.detach()
            str_h = 'const float conv_' + str(cov_cnt) + '_b' + '[' + str(b.size()[0]) + ']' + ' = \n' 
            str_b = str(b)
            str_b = str_b.replace('tensor(', '')
            str_b = str_b.replace(')', '')
            str_b = str_b.replace('[', '{')
            str_b = str_b.replace(']', '}')
            str1 = str_h + str_b + ';\n'
            cov_cnt = cov_cnt + 1    
            config_file.write(str1)
        elif type(sub_module) is torch.nn.Linear:
            w = sub_module.weight.detach()
            str_h = 'const float linear_' + str(linear_cnt) + '_w' + '[' + str(w.size()[0]) + ']'  \
            + '[' + str(w.size()[1]) + ']' + ' = \n' 
            str_b = str(w)
            str_b = str_b.replace('tensor(', '')
            str_b = str_b.replace(')', '')
            str_b = str_b.replace('[', '{')
            str_b = str_b.replace(']', '}')
            str1 = str_h + str_b + ';\n'
            cov_cnt = cov_cnt + 1    
            config_file.write(str1)
            
            w = sub_module.bias.detach()
            str_h = 'const float linear_' + str(linear_cnt) + '_b' + '[' + str(w.size()[0]) + ']' + ' = \n' 
            str_b = str(w)
            str_b = str_b.replace('tensor(', '')
            str_b = str_b.replace(')', '')
            str_b = str_b.replace('[', '{')
            str_b = str_b.replace(']', '}')
            str1 = str_h + str_b + ';\n'
            cov_cnt = cov_cnt + 1    
            config_file.write(str1)
            linear_cnt = linear_cnt + 1
    
    config_file.close()
generate_params(model, 'params.h')